In [104]:
import re
import time
import math
import urllib.parse
import requests
from tqdm import tqdm  # status bars
import pandas as pd

# --------------------------------------------------------
# USER INPUT: file paths & parameters
# --------------------------------------------------------
LOCATIONS_FILE = "locations.tsv"  # your uploaded locations file
SPECIES_FILE = "species.tsv"      # your uploaded species file
OUTPUT_FILE = "analysis_results.tsv"

MAX_OBIS_RECORDS = 10000          # how many OBIS occurrences to consider
MAX_REGION_DIAGONAL_KM = 3000    # filter out marine regions larger than this


# --------------------------------------------------------
# CONSTANTS
# --------------------------------------------------------
BASE_OBIS = "https://api.obis.org/v3"
BASE_MARINE_REGIONS = "https://www.marineregions.org/rest"
BASE_WORMS = "https://www.marinespecies.org/rest"


# ========================================================
# 0. Small helpers
# ========================================================
def dms_to_decimal(dms_str):
    """
    Convert a coordinate in DMS format like '51°06'39.3"N'
    or '2°36'13.6"E' to decimal degrees.
    """
    if not isinstance(dms_str, str):
        raise ValueError(f"Invalid DMS value: {dms_str}")

    # matches: degrees, minutes, seconds, hemisphere (N/S/E/W)
    m = re.match(
        r"^\s*(\d+)[°:\s]+(\d+)[\'’:\s]+(\d+(?:\.\d+)?)[\"”]?\s*([NSEW])\s*$",
        dms_str.strip()
    )
    if not m:
        raise ValueError(f"Cannot parse DMS: {dms_str}")

    deg, mins, secs, hemi = m.groups()
    deg = float(deg)
    mins = float(mins)
    secs = float(secs)

    decimal = deg + mins / 60.0 + secs / 3600.0
    if hemi in ("S", "W"):
        decimal = -decimal
    return decimal


# ========================================================
# 1. OBIS helpers: occurrences + sea-only distance
# ========================================================
def get_obis_occurrences(scientific_name, limit=1000):
    """
    Retrieve occurrence coordinates for a given species from OBIS.
    Returns a list of dicts with:
      - decimalLatitude
      - decimalLongitude
      - occurrenceID (OBIS occurrence id, if present)
    """
    url = f"{BASE_OBIS}/occurrence"
    params = {
        "scientificname": scientific_name,
        "size": limit
    }
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()

    records = []
    for rec in data.get("results", []):
        lat = rec.get("decimalLatitude")
        lon = rec.get("decimalLongitude")
        if lat is None or lon is None:
            continue
        records.append({
            "decimalLatitude": float(lat),
            "decimalLongitude": float(lon),
            "occurrenceID": rec.get("id")
        })
    return records


def sea_distance_km(ref_lat, ref_lon, lat, lon):
    """
    Shortest sea route distance in km using searoute.

    Requires:
      pip install searoute
    """
    import searoute as sr  # imported here to avoid dependency if not needed

    origin = [ref_lon, ref_lat]     # searoute expects [lon, lat]
    destination = [lon, lat]

    route = sr.searoute(origin, destination, units="km")
    if route is None:
        return None
    return float(route.properties["length"])


def find_closest_obis_occurrence(scientific_name, new_lat, new_lon, limit=200):
    """
    From all OBIS occurrences of 'scientific_name', find the one with
    the shortest sea-only distance to (new_lat, new_lon).
    Shows a tqdm progress bar for the sea-routing.
    """
    occurrences = get_obis_occurrences(scientific_name, limit=limit)
    if not occurrences:
        raise RuntimeError(
            f"No OBIS occurrences with coordinates found for '{scientific_name}'."
        )

    best = None

    # Progress bar over all OBIS records
    for rec in tqdm(occurrences,
                    desc=f"Sea-routing OBIS for {scientific_name}",
                    unit="record",
                    leave=False):
        lat = rec["decimalLatitude"]
        lon = rec["decimalLongitude"]
        try:
            d = sea_distance_km(new_lat, new_lon, lat, lon)
        except Exception as e:
            print(f"[WARN] sea routing failed for ({lat}, {lon}): {e}")
            d = None

        rec["sea_distance_km"] = d
        if d is None:
            continue

        if best is None or d < best["sea_distance_km"]:
            best = rec

    if best is None:
        raise RuntimeError(
            f"No valid sea-only distances could be computed to OBIS occurrences "
            f"for '{scientific_name}'."
        )

    return best


# ========================================================
# 2. Marine Regions helpers (with bbox size filter)
# ========================================================

# cache for MRGID -> diagonal distance (km)
_region_diag_cache = {}


def bbox_diagonal_km_from_degrees(min_lat, min_lon, max_lat, max_lon):
    """
    Approximate diagonal distance (km) of a lat/lon bounding box
    using simple degree-to-km conversions.
    """
    if not (-90 <= min_lat <= 90 and -90 <= max_lat <= 90):
        return None
    if not (-180 <= min_lon <= 180 and -180 <= max_lon <= 180):
        return None

    dlat_deg = max_lat - min_lat
    dlon_deg = max_lon - min_lon

    # Normalize longitude difference to [-180, 180]
    if dlon_deg > 180:
        dlon_deg -= 360
    elif dlon_deg < -180:
        dlon_deg += 360

    lat_km_per_deg = 111.32
    mid_lat = (min_lat + max_lat) / 2.0
    lon_km_per_deg = 111.32 * math.cos(math.radians(mid_lat))

    dlat_km = dlat_deg * lat_km_per_deg
    dlon_km = dlon_deg * lon_km_per_deg

    diag_km = math.sqrt(dlat_km**2 + dlon_km**2)

    if diag_km > 25000:
        return None

    return diag_km


def get_region_bbox_diagonal_km(mrgid):
    """
    Get approximate diagonal (km) of the bounding box for a Marine Region (MRGID).
    Uses a small cache to avoid repeated calls for the same MRGID.
    """
    mrgid_str = str(mrgid)
    if mrgid_str in _region_diag_cache:
        return _region_diag_cache[mrgid_str]

    url = f"{BASE_MARINE_REGIONS}/getGazetteerRecordByMRGID.json/{mrgid_str}/"

    try:
        resp = requests.get(url, headers={"accept": "application/json"}, timeout=15)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"[WARN] Request error for MRGID {mrgid_str}: {e}")
        _region_diag_cache[mrgid_str] = None
        return None

    try:
        data = resp.json()
    except ValueError:
        print(f"[WARN] Could not decode JSON for MRGID {mrgid_str}")
        _region_diag_cache[mrgid_str] = None
        return None

    # API may return a list or a dict
    if isinstance(data, list):
        if not data:
            _region_diag_cache[mrgid_str] = None
            return None
        data = data[0]

    if not isinstance(data, dict):
        _region_diag_cache[mrgid_str] = None
        return None

    min_lat = data.get("minLatitude")
    min_lon = data.get("minLongitude")
    max_lat = data.get("maxLatitude")
    max_lon = data.get("maxLongitude")

    if None in (min_lat, min_lon, max_lat, max_lon):
        _region_diag_cache[mrgid_str] = None
        return None

    try:
        min_lat = float(min_lat)
        min_lon = float(min_lon)
        max_lat = float(max_lat)
        max_lon = float(max_lon)
    except ValueError:
        _region_diag_cache[mrgid_str] = None
        return None

    diag_km = bbox_diagonal_km_from_degrees(min_lat, min_lon, max_lat, max_lon)
    _region_diag_cache[mrgid_str] = diag_km
    return diag_km


def get_filtered_mrgids_for_coord(lat_dd, lon_dd, max_diagonal_km=3000):
    """
    Get Marine Regions MRGIDs for a point and filter out regions whose bounding
    box diagonal is more than max_diagonal_km.

    Returns a list of MRGID strings that:
      - have bounding box info, AND
      - diagonal distance <= max_diagonal_km
    """
    try:
        lat = float(lat_dd)
        lon = float(lon_dd)
    except (TypeError, ValueError):
        return []

    url = f"{BASE_MARINE_REGIONS}/getGazetteerRecordsByLatLong.json/{lat}/{lon}/"

    try:
        resp = requests.get(url, headers={"accept": "application/json"}, timeout=15)
        resp.raise_for_status()
        regions = resp.json()
    except Exception as e:
        print(f"[WARN] Marine Regions request failed for {lat}, {lon}: {e}")
        return []

    if not regions or not isinstance(regions, list):
        return []

    filtered_mrgids = []

    for item in regions:
        mrgid = item.get("MRGID")
        if not mrgid:
            continue

        diag_km = get_region_bbox_diagonal_km(mrgid)

        # Keep only if bbox is known AND size <= threshold
        if diag_km is not None and diag_km <= max_diagonal_km:
            filtered_mrgids.append(str(mrgid))

    return filtered_mrgids


# ========================================================
# 3. WoRMS/WRiMS helpers
# ========================================================

aphia_cache = {}
dist_cache = {}


def get_aphia_id(scientific_name, marine_only=True):
    """
    Use WoRMS REST:
      GET /AphiaIDByName/{ScientificName}?marine_only=true

    Returns AphiaID (int) or None
    """
    if scientific_name in aphia_cache:
        return aphia_cache[scientific_name]

    encoded_name = urllib.parse.quote(scientific_name)
    url = f"{BASE_WORMS}/AphiaIDByName/{encoded_name}"

    params = {}
    if marine_only:
        params["marine_only"] = "true"

    try:
        r = requests.get(url, params=params, timeout=20)
        if r.status_code == 204 or not r.text.strip():
            aphia_cache[scientific_name] = None
            return None
        r.raise_for_status()
        text = r.text.strip()
        if text == "0":
            aphia_cache[scientific_name] = None
            return None
        aphia_id = int(text)
    except Exception as e:
        print(f"[WARN] Could not get AphiaID for '{scientific_name}': {e}")
        aphia_cache[scientific_name] = None
        return None

    aphia_cache[scientific_name] = aphia_id
    time.sleep(0.1)
    return aphia_id


def get_distributions_for_aphia(aphia_id):
    """
    Use WoRMS REST:
      GET /AphiaDistributionsByAphiaID/{ID}

    Returns a list of dicts.
    """
    if aphia_id in dist_cache:
        return dist_cache[aphia_id]

    url = f"{BASE_WORMS}/AphiaDistributionsByAphiaID/{aphia_id}"

    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 204 or not r.text.strip():
            dist_cache[aphia_id] = []
            return []
        r.raise_for_status()
        data = r.json()
        if not isinstance(data, list):
            data = []
    except Exception as e:
        print(f"[WARN] Could not get distributions for AphiaID {aphia_id}: {e}")
        dist_cache[aphia_id] = []
        return []

    dist_cache[aphia_id] = data
    time.sleep(0.1)
    return data


def extract_mrgid_from_locationID(locationID):
    """
    distribution['locationID'] looks like:
      'http://marineregions.org/mrgid/7130'
    We want the numeric MRGID as string, e.g. '7130'.
    """
    if not isinstance(locationID, str):
        return None
    m = re.search(r"/mrgid/(\d+)", locationID)
    return m.group(1) if m else None


def classify_invasiveness(scientific_name, mrgids_for_location):
    """
    Returns dict with:
      wrims_has_record, wrims_invasive, wrims_establishment, wrims_matching_mrgid
    """
    if not mrgids_for_location:
        return {
            "wrims_has_record": False,
            "wrims_invasive": None,
            "wrims_establishment": None,
            "wrims_matching_mrgid": None
        }

    aphia_id = get_aphia_id(scientific_name)
    if aphia_id is None:
        return {
            "wrims_has_record": False,
            "wrims_invasive": None,
            "wrims_establishment": None,
            "wrims_matching_mrgid": None
        }

    distribs = get_distributions_for_aphia(aphia_id)
    if not distribs:
        return {
            "wrims_has_record": False,
            "wrims_invasive": None,
            "wrims_establishment": None,
            "wrims_matching_mrgid": None
        }

    # Build index: MRGID -> list of distribution entries
    mrgid_to_records = {}
    for d in distribs:
        loc_id = d.get("locationID")
        mrgid = extract_mrgid_from_locationID(loc_id)
        if not mrgid:
            continue
        mrgid_to_records.setdefault(mrgid, []).append(d)

    # Check if any of the location's MRGIDs appears in the distribution records
    for loc_mrgid in mrgids_for_location:
        recs = mrgid_to_records.get(loc_mrgid)
        if not recs:
            continue

        best_invasive = None
        best_establishment = None

        for d in recs:
            invasiveness = (d.get("invasiveness") or "").lower()
            establishment = (d.get("establishmentMeans") or "").lower()

            if "invasive" in invasiveness:
                best_invasive = "invasive"
                best_establishment = establishment or None
                break

            if any(word in establishment for word in
                   ["introduced", "alien", "non-indigenous", "nonindigenous"]):
                best_invasive = "introduced"
                best_establishment = establishment or "introduced"

        if best_invasive is not None:
            return {
                "wrims_has_record": True,
                "wrims_invasive": best_invasive,
                "wrims_establishment": best_establishment,
                "wrims_matching_mrgid": loc_mrgid,
            }

    # there are distributions, but nothing matching those MRGIDs with invasiveness info
    return {
        "wrims_has_record": True,
        "wrims_invasive": None,
        "wrims_establishment": None,
        "wrims_matching_mrgid": None
    }


def wrims_label(status_dict):
    """
    Human-friendly label.
    """
    if not status_dict["wrims_has_record"]:
        return "no WRiMS distribution record"

    inv = status_dict["wrims_invasive"]
    if inv == "invasive":
        return "probably invasive"
    if inv == "introduced":
        return "probably introduced"

    return "unknown status (possibly native)"


def classify_with_progress(scientific_name, locations):
    """
    Run WRiMS classification for each (lat, lon) in `locations`
    with a tqdm progress bar.

    locations: dict[label] = (lat, lon)

    Returns dict[label] = {
        "mrgids": [...],
        "status": {...},
        "label": "human-friendly label"
    }
    """
    results = {}

    for key in tqdm(locations.keys(),
                    desc=f"WRiMS status for {scientific_name}",
                    unit="location",
                    leave=False):
        lat, lon = locations[key]
        mrgids = get_filtered_mrgids_for_coord(
            lat, lon, max_diagonal_km=MAX_REGION_DIAGONAL_KM
        )
        status = classify_invasiveness(scientific_name, mrgids)
        results[key] = {
            "mrgids": mrgids,
            "status": status,
            "label": wrims_label(status)
        }

    return results


# ========================================================
# 4. MAIN FLOW: batch over TSV files
# ========================================================
def main():
    # 4.1 Read input TSVs
    print(f"Reading locations from {LOCATIONS_FILE}")
    locations = pd.read_csv(LOCATIONS_FILE, sep="\t")

    print(f"Reading species from {SPECIES_FILE}")
    species = pd.read_csv(SPECIES_FILE, sep="\t")

    # 4.2 Convert DMS to decimal degrees
    print("Converting coordinates to decimal degrees...")
    locations["decimalLatitude"] = locations["verbatimLatitude"].apply(dms_to_decimal)
    locations["decimalLongitude"] = locations["verbatimLongitude"].apply(dms_to_decimal)

    # 4.3 Join species with locations via locationID
    merged = species.merge(locations, on="locationID", how="inner")

    print(f"Total (location, species) cases to analyse: {len(merged)}")

    results = []

    # 4.4 Loop over each row and run your full analysis
    for _, row in tqdm(merged.iterrows(),
                       total=len(merged),
                       desc="Total analyses",
                       unit="case"):
        loc_id = row["locationID"]
        verb_lat = row["verbatimLatitude"]
        verb_lon = row["verbatimLongitude"]
        lat = float(row["decimalLatitude"])
        lon = float(row["decimalLongitude"])
        sci_name = row["scientificName"]
        taxon_rank = row.get("taxonRank", "")

        # ---- OBIS: closest occurrence by sea-only distance ----
        try:
            closest = find_closest_obis_occurrence(
                sci_name,
                lat,
                lon,
                limit=MAX_OBIS_RECORDS
            )
            obis_lat = closest["decimalLatitude"]
            obis_lon = closest["decimalLongitude"]
            obis_dist_km = closest["sea_distance_km"]
        except Exception as e:
            print(f"[WARN] OBIS step failed for {sci_name} at {loc_id}: {e}")
            closest = None
            obis_lat = None
            obis_lon = None
            obis_dist_km = None

        # ---- WRiMS classification for new & closest OBIS locations ----
        locations_dict = {
            "new_observation": (lat, lon)
        }
        if closest is not None:
            locations_dict["closest_obis"] = (obis_lat, obis_lon)

        wrims_results = classify_with_progress(sci_name, locations_dict)

        new_label = wrims_results["new_observation"]["label"]
        obis_label = wrims_results.get("closest_obis", {}).get("label")

        # ---- Collect row for output ----
        results.append({
            "locationID": loc_id,
            "verbatimLatitude": verb_lat,
            "verbatimLongitude": verb_lon,
            "decimalLatitude": lat,
            "decimalLongitude": lon,
            "scientificName": sci_name,
            "taxonRank": taxon_rank,
            "closest_obis_lat": obis_lat,
            "closest_obis_lon": obis_lon,
            "distance_over_water_km": obis_dist_km,
            "wrims_status_new_location": new_label,
            "wrims_status_closest_obis": obis_label,
        })

    # 4.5 Write single TSV output
    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_FILE, sep="\t", index=False)
    print(f"\nDone. Results written to: {OUTPUT_FILE}")


if __name__ == "__main__":
    main()


Reading locations from locations.tsv
Reading species from species.tsv
Converting coordinates to decimal degrees...
Total (location, species) cases to analyse: 50


Sea-routing OBIS for Acartia tonsa:   0%| | 50/10000 [00:00<00:20, 493.86record/s]
Sea-routing OBIS for Acartia tonsa:   1%| | 106/10000 [00:00<00:18, 527.07record/s
Sea-routing OBIS for Acartia tonsa:   2%| | 163/10000 [00:00<00:18, 540.13record/s
Sea-routing OBIS for Acartia tonsa:   2%| | 218/10000 [00:00<00:18, 517.93record/s
Sea-routing OBIS for Acartia tonsa:   3%| | 284/10000 [00:00<00:17, 555.15record/s
Sea-routing OBIS for Acartia tonsa:   4%| | 360/10000 [00:00<00:15, 611.93record/s
Sea-routing OBIS for Acartia tonsa:   4%| | 422/10000 [00:00<00:16, 586.15record/s
Sea-routing OBIS for Acartia tonsa:   5%| | 490/10000 [00:00<00:15, 609.28record/s
Sea-routing OBIS for Acartia tonsa:   6%| | 552/10000 [00:00<00:17, 551.94record/s
Sea-routing OBIS for Acartia tonsa:   6%| | 618/10000 [00:01<00:16, 575.74record/s
Sea-routing OBIS for Acartia tonsa:   7%| | 677/10000 [00:01<00:17, 531.14record/s
Sea-routing OBIS for Acartia tonsa:   7%| | 732/10000 [00:01<00:17, 525.91record/s
Sea-

[WARN] OBIS step failed for Cryptorchestia garbinii at Site A: No OBIS occurrences with coordinates found for 'Cryptorchestia garbinii'.



WRiMS status for Cryptorchestia garbinii:   0%|       | 0/1 [00:00<?, ?location/s]
WRiMS status for Cryptorchestia garbinii: 100%|█| 1/1 [00:01<00:00,  1.29s/locatio
Sea-routing OBIS for Dasysiphonia japonica:   0%|    | 0/1728 [00:00<?, ?record/s]
Sea-routing OBIS for Dasysiphonia japonica:   2%| | 26/1728 [00:00<00:06, 259.29re
Sea-routing OBIS for Dasysiphonia japonica:   5%| | 88/1728 [00:00<00:03, 454.20re
Sea-routing OBIS for Dasysiphonia japonica:   8%| | 133/1728 [00:00<00:04, 397.20r
Sea-routing OBIS for Dasysiphonia japonica:  10%| | 180/1728 [00:00<00:03, 422.77r
Sea-routing OBIS for Dasysiphonia japonica:  13%|▏| 227/1728 [00:00<00:03, 437.36r
Sea-routing OBIS for Dasysiphonia japonica:  16%|▏| 272/1728 [00:00<00:03, 368.63r
Sea-routing OBIS for Dasysiphonia japonica:  19%|▏| 329/1728 [00:00<00:03, 424.14r
Sea-routing OBIS for Dasysiphonia japonica:  22%|▏| 374/1728 [00:00<00:03, 417.54r
Sea-routing OBIS for Dasysiphonia japonica:  25%|▏| 425/1728 [00:01<00:02, 436.82r
Sea


Done. Results written to: analysis_results.tsv
